# Assignment 1: Dino Fun World

### Assignment Description

You, in your role as a data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a few questions about their park operations. The dataset that they provided for you to perform the requested analysis includes the movement and communication data captured from the park attendees' apps during one weekend (Friday, Saturday, and Sunday).

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.
* **Question 1:** What is the most popular attraction to visit in the park?


* **Question 2:** What ride (note that not all attractions are rides) has the longest average visit time?


* **Question 3:** Which Fast Food offering has the fewest visitors?


* **Question 4:** Compute the Skyline of number of visits and visit time for the park's ride, and report the rides that appear in the Skyline. (Note: Your answer should be three points, which can be given in any order.)


### Directions

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkin', 'attractions', and 'sequences'. The database file is named 'dinofunworld.db' .

The information contained in each of these tables is listed below:

`checkin`:
    - The check-in data for all visitors for the day in the park. The data includes two types of check-ins: inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type

`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carousel, etc.
    - Fields: AttractionID, Name, Region, Category, type
    
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
Using the provided data, answer the four questions that the administrators have asked.

### Submission Directions for Assignment Deliverables

This assignment will be auto-graded. We recommend that you submit this assignment. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. 

Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed. 

Please execute each cell in Jupyter Notebook before submitting.




### Question 1: 
What is the most popular attraction to visit in the park?

**Note:** Your output should be the name of the attraction.

In [1]:
# your code here for Q1
import sqlite3
import pandas as pd

# Establishing connection to database
db_con = sqlite3.connect("dinofunworld.db")
# Setting up the cursor to access values of the table
pointer = db_con.cursor()
# Querying the checkin table to get attraction count
pointer.execute("SELECT attraction, count(*) FROM checkin GROUP BY attraction")
att_count = pointer.fetchall()
# Building the dataframe with columns attraction and count
att_count = pd.DataFrame(att_count, columns = ['attractionid', 'count'])
# Fetches the row of the max count value
att_count_max = att_count.loc[att_count['count'].idxmax()]
att_id = att_count_max['attractionid']
pointer.execute("SELECT AttractionID, Name FROM attraction")

att_name = pointer.fetchall()
# iterating throught the list formed to find popular attraction to visit in the park
for id, name in att_name:
    if id==att_id:
        print(name)
        
# Close the connection
db_con.close()

Atmosfear


### Question 2: 

What ride (note that not all attractions are rides) has the  longest average visit time?

**Note:** Your output should be the name of the ride.


In [2]:
# your code here for Q2
import sqlite3
from datetime import timedelta

# Connect to the database
db_con = sqlite3.connect('dinofunworld.db')
# Setting up the cursor to access values of the table
pointer = db_con.cursor()

# Get the AttractionID of the rides from the 'attractions' table
pointer.execute("SELECT AttractionID FROM attraction WHERE Category={0} OR Category={1} OR Category={2}".format("'Thrill Rides\r'", "'Kiddie Rides\r'", "'Rides for Everyone\r'"))
list_ride = [i[0] for i in pointer.fetchall()]

# Iterate over each i and calculate the average duration
ride_map = list()
for i in list_ride:
    ride_time = 0.0
    count = 0
    # Getting the duration of the rides from checkin table
    pointer.execute("SELECT duration FROM checkin WHERE attraction={0}".format(i))
    # iterating through the durations list
    for duration in pointer.fetchall():
        try:
            # Convert the duration string to seconds and add to the total i duration
            ride_time += timedelta(int(duration[0].split(':')[0]), int(duration[0].split(':')[1]), int(duration[0].split(':')[2])).total_seconds()
            count += 1
        except (ValueError, AttributeError) as e:
            pass
        #Mapping the duration to the i
    ride_map.append([i, ride_time / count])

# Get the name of the i with the longest average visit time
pointer.execute("SELECT Name FROM attraction WHERE AttractionID={0}".format(max(ride_map, key=lambda item:item[1])[0]))

# Fetch and print the result
res = pointer.fetchone()
print(res[0])

# Close the connection
db_con.close()

Flight of the Swingodon


### Question 3:

Which Fast Food offering in the park has the fewest visitors?

**Note:** Your output should be the name of the fast food offering.

In [3]:
# your code here for Q3
import sqlite3
import pandas as pd
# Establishing connection to database
db_con = sqlite3.connect("dinofunworld.db")
# Setting up the cursor to access values of the table
pointer = db_con.cursor()
# Querying the attraction table to get attractionid, name and type
pointer.execute("SELECT AttractionID, Name, type FROM attraction")
# Creating dataframe for the list 
att_df =  pd.DataFrame(pointer.fetchall(), columns = ['attractionid','name','type'])
# finding out the ids which has tyoe as "Fast food"
fast_food_df = att_df.loc[att_df['type']=='Fast Food']
# Executing the query by grouping attraction in checkin table
pointer.execute("SELECT attraction,count(*) FROM checkin GROUP BY attraction")
# creating dataframe of the list having attractionid and their count
att_count = pd.DataFrame(pointer.fetchall(), columns = ['attractionid','count'])
merger_df = pd.merge(fast_food_df, att_count, left_on = 'attractionid', right_on = 'attractionid', how = 'inner').drop('attractionid', axis = 1)
#finding the min value in the column count
min_count=merger_df.loc[merger_df['count'].idxmin()]
print(min_count['name'])
# Close the connection
db_con.close()

Theresaur Food Stop


### Question 4:

Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline. 

**Note:** Remember that in this case, higher visits is better and lower visit times are better. Your output should be formatted as a list of names of the rides in the Skyline. Your output should be three points, which can be given in any order(example output : ['Ride 1','Ride 2','Ride 3'] )


In [4]:
# your code here for Q4
from datetime import timedelta

# Connect to the database
db_con = sqlite3.connect('dinofunworld.db')
# Setting up the pointer to access values of the table
pointer = db_con.cursor()

# Get the AttractionID of the rides from the 'attractions' table
pointer.execute("SELECT AttractionID FROM attraction WHERE Category={0} OR Category={1} OR Category={2}".format("'Thrill Rides\r'", "'Kiddie Rides\r'", "'Rides for Everyone\r'"))
list_ride = [i[0] for i in pointer.fetchall()]

# Create a dictionary to store the average duration and number of visits for each i
ride_map = dict()
# iterating the dictonary
for i in list_ride:
    ride_time = 0.0
    count = 0.0
    # Getting the duration of the rides from checkin table
    pointer.execute("SELECT duration FROM checkin WHERE attraction={0}".format(i))
    for duration in pointer.fetchall():
        try:
            # Convert the duration string to seconds and add to the total i duration
            ride_time += timedelta(hours=int(duration[0].split(':')[0]), minutes=int(duration[0].split(':')[1]), seconds=int(duration[0].split(':')[2])).total_seconds()
            count += 1
        except (ValueError, AttributeError) as e:
            pass
    ride_map[i] = list()
    ride_map[i].append(ride_time / count)
    # Getting the total count from the checkin table for the given i
    pointer.execute("SELECT count(*) FROM checkin WHERE attraction={0}".format(i))
    ride_map[i].append(pointer.fetchall()[0][0])

# Compute the Skyline of the rides
rideV = 1
rideW = 1
rideB = 1

# Iterating the dictonary for the condition 
for i in ride_map:
    if ride_map[i][0] < ride_map[rideW][0]:
        rideW = i
    if ride_map[i][1] > ride_map[rideV][1]:
        rideV = i
    if (ride_map[i][0] < ride_map[rideB][0]) and (ride_map[i][1] > ride_map[rideB][1]):
        rideB = i

# Get the names of the rides in the Skyline
pointer.execute("SELECT Name FROM attraction WHERE AttractionID={0} OR AttractionID={1} OR AttractionID={2}".format(rideV, rideW, rideB))
# Fetching the table
names = pointer.fetchall()
print([name[0] for name in names])

# Close the connection
db_con.close()


['Wrightiraptor Mountain', 'Atmosfear', 'Dykesadactyl Thrill']
